In [2]:
# General imports

import numpy as np
import pandas as pd
import gzip
import os, sys, gc, warnings, random, datetime, math, awscli

from sklearn import metrics
from sklearn.model_selection import train_test_split, KFold,GroupShuffleSplit
from sklearn.preprocessing import LabelEncoder

import seaborn as sns
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')

In [ ]:
# Helper functions



In [6]:
# Run this once to download the CSV files to your local ./data/csv_files directory. 
# Make sure you have configured your AWS Credential File
# Make sure you have a created a /data/cvs_files folder
!aws s3 sync s3://sagemaker-us-east-1-936165954724/ml-turnover/ ./data/csv_files

In [82]:
### LOAD FIRM-LEVEL SHARE PRICE DATA

def load_performance_data():
    """
    
    Load a firm-level share price performance dataset
    Return a DataFrame with price closing monthly (prccm)

    Parameters
    ----------
    csv_path : str with location of csv file
    s3 : bool, default False
        Whether to use a csv file stored in an S3 location

    Returns
    -------
    pandas.DataFrame:
        MultiIndex:
            gvkey, Integer, dtype: int64
            date: Date, dtype: datetime64[M]
        Columns:
            company_name: String, dtype: string
            prccm: Float, dtype: float64
            industry_sic_code: Integer, dtype: int64
            industry_naics_code: Integer, dtype: int64
    """
    
    csv_file = './data/csv_files/share_prices.csv.gz'
    df = pd.read_csv(csv_file, compression='gzip')
    df.datadate = pd.to_datetime(df.datadate, format="%Y%m%d")
    
    return df

df_share_prices = load_performance_data() 

In [26]:
### LOAD FIRM-LEVEL SHARE PRICE DATA

def load_index_data():
    """
    
    Load a firm-level share price performance dataset
    Return a DataFrame with price closing monthly (prccm)

    Parameters
    ----------
    csv_path : str with location of csv file
    s3 : bool, default False
        Whether to use a csv file stored in an S3 location

    Returns
    -------
    pandas.DataFrame:
        MultiIndex:
            gvkey, Integer, dtype: int64
            date: Date, dtype: datetime64[M]
        Columns:
            company_name: String, dtype: string
            prccm: Float, dtype: float64
            industry_sic_code: Integer, dtype: int64
            industry_naics_code: Integer, dtype: int64
    """
    
    csv_file = './data/csv_files/index_prices.csv.gz'
    df = pd.read_csv(csv_file, compression='gzip')
    df.datadate = pd.to_datetime(df.datadate, format="%Y%m%d")
    
    return df

df_index_prices = load_index_data() 

In [27]:
df_index = df_index_prices[df_index_prices["conm"] == "Russell 3000"]

In [28]:
# Drop all columsn with "NA" values

df_index = df_index[["datadate", "prccm"]].rename(columns={"prccm":"price"})

In [29]:
def calculate_index_returns(df, periods=[1, 3, 12, 24, 36]):
    """
    Return dataframe with monthly returns
    
    Parameters
    ----------
    df : pandas.DataFrame
    periods : array of Integer
        Integers to calculate percentage return, 
        i.e. 12 --> (t0 - t-12)/(t-12)

    Returns
    -------
    pandas.DataFrame:
        MultiIndex:
            gvkey, Integer, dtype: int64
            date: Date, dtype: datetime64[M]
        Columns:
            company_name: String, dtype: string
            prccm: Float, dtype: float64
            industry_sic_code: Integer, dtype: int64
            industry_naics_code: Integer, dtype: int64
            pct_change_{period_1}: Float, dtype: float64
            pct_change_{period_x}: Float, dtype: float64
    """

    for period in periods:
        key_1 = 'return_' + str(period) + 'M'
        key_2 = 'annualized_return_' + str(period) + 'M'
        # use ffill method to deal with missing closing prices data
        df[key_1] = df['price'].pct_change(fill_method='ffill', periods=period)
        df[key_2] = ((df[key_1]+1)**(12/period)-1)
    
    return df

df_index = calculate_index_returns(df_index)

In [56]:
def check_nans(df):
    nans_df = df.isna()
    nans_groups={}

    for col in df.columns:
        cur_group = nans_df[col].sum()
        if cur_group >= 0:
            print(f'{col}')
            print(f'NAN row count = {cur_group}')
            print(f'% NAN count = {df[col].isnull().mean()*100:.2f}%\n')
        
    del nans_df; x=gc.collect()

check_nans(df_share_prices)

gvkey
NAN row count = 0
% NAN count = 0.00%

iid
NAN row count = 0
% NAN count = 0.00%

datadate
NAN row count = 0
% NAN count = 0.00%

tic
NAN row count = 533
% NAN count = 0.01%

cusip
NAN row count = 0
% NAN count = 0.00%

conm
NAN row count = 0
% NAN count = 0.00%

ajexm
NAN row count = 37923
% NAN count = 0.69%

ajpm
NAN row count = 37923
% NAN count = 0.69%

prccm
NAN row count = 93009
% NAN count = 1.70%

trfm
NAN row count = 75471
% NAN count = 1.38%

city
NAN row count = 4327
% NAN count = 0.08%

naics
NAN row count = 117695
% NAN count = 2.15%

sic
NAN row count = 4356
% NAN count = 0.08%

state
NAN row count = 547344
% NAN count = 9.99%



In [54]:
# Drop columns with more than 10% missing values
df_share_prices = df_share_prices.loc[:, df_share_prices.isnull().mean() < .1]

In [107]:
the_list = df_share_prices[df_share_prices['ajexm'] == 0]["gvkey"].unique()

In [112]:
df_share_prices = df_share_prices[~df_share_prices["gvkey"].isin(the_list)]

In [123]:
# calculate the stock price adjusted for splits
df_share_prices["price"] = df_share_prices["prccm"]/df_share_prices["ajexm"] 

In [ ]:
# let's drop 

In [152]:
# df_share_prices[df_share_prices["return_period_36"] == np.inf]
df_share_prices[df_share_prices["tic"].na()][""]
# df_share_prices.columns
# df_share_prices[df_share_prices["tick"] == 2662][350:389][["tic", "gvkey", "datadate", "conm", "ajexm", "prccm", "price"]]

,gvkey,iid,datadate,tic,cusip,conm,ajexm,ajpm,dvpspm,dvpsxm,...,gsubind,naics,sic,state,price,return_period_1,return_period_3,return_period_12,return_period_24,return_period_36
118074,2433,03,2001-05-31,NaN,055961403,BNS HOLDING CO,1.0,1.0,15.25,15.25,...,20106020.0,336211.0,3713.0,RI,NaN,0.000000,0.000000,0.000000,0.393443,0.923077
1152729,14236,01,2012-11-30,NaN,24099G932,MANSFIELD-MARTIN EN MG -OLD,1.0,1.0,NaN,NaN,...,15104040.0,212221.0,1040.0,AZ,0.150,NaN,NaN,NaN,NaN,NaN
1152730,14236,01,2012-12-31,NaN,24099G932,MANSFIELD-MARTIN EN MG -OLD,1.0,1.0,NaN,NaN,...,15104040.0,212221.0,1040.0,AZ,0.110,-0.266667,NaN,NaN,NaN,NaN
1152731,14236,01,2013-01-31,NaN,24099G932,MANSFIELD-MARTIN EN MG -OLD,1.0,1.0,NaN,NaN,...,15104040.0,212221.0,1040.0,AZ,0.110,0.000000,NaN,NaN,NaN,NaN
1152732,14236,01,2013-02-28,NaN,24099G932,MANSFIELD-MARTIN EN MG -OLD,1.0,1.0,NaN,NaN,...,15104040.0,212221.0,1040.0,AZ,0.200,0.818182,0.333333,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5459493,277891,01,2019-12-31,NaN,G23474102,CONDOR GOLD PLC,1.0,1.0,NaN,NaN,...,15104030.0,212221.0,1040.0,NaN,0.310,0.377778,-0.138889,0.033333,-0.578231,NaN
5459494,277891,01,2020-01-31,NaN,G23474102,CONDOR GOLD PLC,1.0,1.0,NaN,NaN,...,15104030.0,212221.0,1040.0,NaN,0.250,-0.193548,0.041667,-0.418605,-0.659864,NaN
5459495,277891,01,2020-02-29,NaN,G23474102,CONDOR GOLD PLC,1.0,1.0,NaN,NaN,...,15104030.0,212221.0,1040.0,NaN,0.320,0.280000,0.422222,-0.255814,-0.466667,NaN
5459496,277891,01,2020-03-31,NaN,G23474102,CONDOR GOLD PLC,1.0,1.0,NaN,NaN,...,15104030.0,212221.0,1040.0,NaN,0.355,0.109375,0.145161,0.221610,-0.556250,NaN


In [ ]:
sns.distplot(df_share_prices["return_period_36"], kde=False, bins=100)

In [ ]:
# Drop rows with no SIC codes

In [126]:
def calculate_returns(df, periods=[1, 3, 12, 24, 36]):
    """
    Return dataframe with monthly returns
    
    Parameters
    ----------
    df : pandas.DataFrame
    periods : array of Integer
        Integers to calculate percentage return, 
        i.e. 12 --> (t0 - t-12)/(t-12)

    Returns
    -------
    pandas.DataFrame:
        MultiIndex:
            gvkey, Integer, dtype: int64
            date: Date, dtype: datetime64[M]
        Columns:
            company_name: String, dtype: string
            prccm: Float, dtype: float64
            industry_sic_code: Integer, dtype: int64
            industry_naics_code: Integer, dtype: int64
            pct_change_{period_1}: Float, dtype: float64
            pct_change_{period_x}: Float, dtype: float64
    """

    for period in periods:
        key_name = 'return_period_' + str(period)
        # use ffill method to deal with missing closing prices data
        df[key_name] = df.groupby('gvkey')['price'].pct_change(fill_method='ffill', periods=period)

    return df

df_share_prices = calculate_returns(df_share_prices)

In [ ]:
# "EXPERT-LEVEL" TO-DO ;-) Optional for Ozzi
def industry_returns(df):
    """
    Calculate industry returns for each period
    Group df by industry, calculate mean pct_change, done.
    
    Parameters
    ----------
    df : pandas.DataFrame

    Returns
    -------
    df: pandas.DataFrame
        Columns:
        ...
        pct_change_{period_1}_industry: Float, dtype: float64
        pct_change_{period_x}_industry: Float, dtype: float64

    """
    
    df['return_period_1_industry_avg'] = df.groupby(['sic', 'date', 'return_period_1']).mean()
    return df

In [37]:
df_share_prices[:100]

In [47]:
df_share_prices.groupby(['sic', 'datadate'])

ValueError: Length mismatch: Expected axis has 5474991 elements, new values have 5479056 elements

In [47]:
########################### TO-DO NICO: DIRECTOR DATA
#################################################################################


def load_director_data():
    csv_file = './data/csv_files/boardex_director_profiles.csv.gz'
    df = pd.read_csv(csv_file, compression='gzip', encoding='latin1')
    # df.datadate = pd.to_datetime(df.datadate, format="%Y%m%d")

    return df

df_directors = load_director_data()


In [68]:
# Drop the unidentfiable names
df_directors = df_directors.iloc[140:, :]

In [73]:

def load_director_age():
    """ Load csv file containing director ages
    
    Args: 
        input_data: csv file in S3 location

    Returns: 
        dataframe with each row corresponding to a single role
    """
    csv_file = './data/csv_files/boardex_director_age_nationality.csv'
    df = pd.read_csv(csv_file, encoding='latin1')
    # df.datadate = pd.to_datetime(df.datadate, format="%Y%m%d")
    return df

df_ages = load_director_age()


In [75]:
df_directors = df_directors.merge(df_ages, on='DirectorID', how='left')


,DirectorName_x,CompanyName,BrdPosition,RoleName,NED,DirectorID,CompanyID,DateStartRole,DateEndRole,HOCountryName,Sector,OrgType,ISIN,DirectorName_y,DOB,Gender,Nationality,NetworkSize
0,A Abedin,AIR LEASE CORP,No,Assistant VP/Treasurer,No,1723369,1698818,20160101,20180501,United States,Transport,Quoted,US00912X3026,A Abedin,n.a.,M,NaN,1282.0
1,A Abedin,AIR LEASE CORP,No,Director - Treasury,No,1723369,1698818,20130801,20151201,United States,Transport,Quoted,US00912X3026,A Abedin,n.a.,M,NaN,1282.0
2,A Abedin,AIR LEASE CORP,No,Executive,No,1723369,1698818,20180501,C,United States,Transport,Quoted,US00912X3026,A Abedin,n.a.,M,NaN,1282.0
3,A Abedin,AIR LEASE CORP,No,Treasurer,No,1723369,1698818,20151201,20160101,United States,Transport,Quoted,US00912X3026,A Abedin,n.a.,M,NaN,1282.0
4,A Abedin,CURTISS-WRIGHT CORP,No,Electrical Engineer,No,1723369,8446,20060601,20100601,United States,Aerospace & Defence,Quoted,US2315611010,A Abedin,n.a.,M,NaN,1282.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5840798,Ümit Suba,SC Johnson Europe Sàrl,No,Regional Director,No,1094972,1677139,20091201,20101201,Switzerland,NaN,Private,NaN,Ümit Suba,1967,M,Turkish,624.0
5840799,Ümit Suba,SC Johnson GmbH,No,Business Director,No,1094972,1677131,19970901,20000601,Germany,NaN,Private,NaN,Ümit Suba,1967,M,Turkish,624.0
5840800,Ümit Suba,SC Johnson GmbH,No,CEO,No,1094972,1677131,20081201,20090901,Germany,NaN,Private,NaN,Ümit Suba,1967,M,Turkish,624.0
5840801,Ümit Suba,SC Johnson GmbH,No,General Manager,No,1094972,1677131,20000701,20040601,Germany,NaN,Private,NaN,Ümit Suba,1967,M,Turkish,624.0


In [107]:
df_small = df_directors[:100]
df_small.index = pd.MultiIndex.from_arrays(df_small[["CompanyID", "DirectorID"]].values.T)
# df_small["hash"] = str(df_small["DirectorID"])+' ' +str(df_small["CompanyID"])
# df_small
# pd.util.hash_pandas_object(df_small[CompanyID]
df_small

DirectorName  \
1698818 1723369     A Abedin   
        1723369     A Abedin   
        1723369     A Abedin   
        1723369     A Abedin   
8446    1723369     A Abedin   
...                      ...   
15995   22951         A Ames   
420450  22951         A Ames   
49567   22951         A Ames   
2397178 22951         A Ames   
49568   22951         A Ames   

                                                      CompanyName BrdPosition  \
1698818 1723369                                    AIR LEASE CORP          No   
        1723369                                    AIR LEASE CORP          No   
        1723369                                    AIR LEASE CORP          No   
        1723369                                    AIR LEASE CORP          No   
8446    1723369                               CURTISS-WRIGHT CORP          No   
...                                                           ...         ...   
15995   22951                                   INFOWAVE SOFTWARE         Yes   
420450  22951                       IPASS INC (De-listed 02/2019)         Yes   
49567   22951                                      Imandi.com Inc         Yes   
2397178 22951    MMGL Corp (Formerly known as Schnitzer West LLC)         Yes   
49568   22951                          MediaOne International Inc         Yes   

                               RoleName  NED  DirectorID  CompanyID  \
1698818 1723369  Assistant VP/Treasurer   No     1723369    1698818   
        1723369     Director - Treasury   No     1723369    1698818   
        1723369               Executive   No     1723369    1698818   
        1723369               Treasurer   No     1723369    1698818   
8446    1723369     Electrical Engineer   No     1723369       8446   
...                                 ...  ...         ...        ...   
15995   22951      Independent Director  Yes       22951      15995   
420450  22951      Independent Director  Yes       22951     420450   
49567   22951             Director - SD  Yes       22951      49567   
2397178 22951             Director - SD  Yes       22951    2397178   
49568   22951             President/CEO   No       22951      49568   

                DateStartRole DateEndRole             HOCountryName  \
1698818 1723369      20160101    20180501             United States   
        1723369      20130801    20151201             United States   
        1723369      20180501           C             United States   
        1723369      20151201    20160101             United States   
8446    1723369      20060601    20100601             United States   
...                       ...         ...                       ...   
15995   22951        20001101    20020101                    Canada   
420450  22951        20020701    20100630             United States   
49567   22951        20000301           N             United States   
2397178 22951               N           C             United States   
49568   22951        19950701    20000601  United Kingdom - England   

                                     Sector  OrgType          ISIN  
1698818 1723369                   Transport   Quoted  US00912X3026  
        1723369                   Transport   Quoted  US00912X3026  
        1723369                   Transport   Quoted  US00912X3026  
        1723369                   Transport   Quoted  US00912X3026  
8446    1723369         Aerospace & Defence   Quoted  US2315611010  
...                                     ...      ...           ...  
15995   22951                           NaN   Quoted           NaN  
420450  22951    Telecommunication Services   Quoted  US46261V1089  
49567   22951                           NaN  Private           NaN  
2397178 22951                           NaN  Private           NaN  
49568   22951                           NaN  Private           NaN  

[100 rows x 13 columns]

In [106]:
def date_conversion(df):
    print(f"Number of rows before processing: {df.shape[0]}")
    df["DOB"] = [np.nan if (val == 'NaN') | (val == 'n.a.') else val for val in df.DOB]
    df.dropna(inplace=True)

    regex = r"(?P<DAY>\d{2})?[/\s-]?(?P<MONTH>[a-zA-Z]{3})?[/\s-]?(?P<YEAR>\d{4})$"
    df = df.join(df["DOB"].str.extract(regex))

    df["DAY"].fillna("01", inplace=True)
    df["MONTH"].fillna("Jan", inplace=True)

    df["DOB"] = df["YEAR"] + "-" + df["MONTH"] + "-" + df["DAY"]
    df["DOB"] = pd.to_datetime(df["DOB"], format="%Y-%b-%d")

    # df.drop(["YEAR", "MONTH", "DAY"], axis=1, inplace=True)
    print(f"Number of rows after dropping NAs: {df.shape[0]}")

    return df

df_small = date_conversion(df_small)

Number of rows before processing: 100


AttributeError: 'DataFrame' object has no attribute 'DOB'

In [ ]:
# Drop the entries that are missing StartDate or EndDate
df.drop(df[df["DateEndRole"] == "N"].index, inplace=True)
df.drop(df[df["DateStartRole"] == "N"].index, inplace=True)

In [ ]:
# Create a RoleId to be used later
df['RoleId'] = df.index.values.tolist()# Create a RoleId to be used later
df['RoleId'] = df.index.values.tolist()

In [ ]:
# Convert "C" DateEndRole to the Maximum End Date in the dataset
MaxDate = np.unique(df["DateEndRole"])[-2]
df.loc[df["DateEndRole"] == "C","DateEndRole"] = MaxDate

In [ ]:
# Create a column to indicate directors who remain in their roles
df["CurrentRole"] = False
df.loc[df["DateEndRole"] == "C","CurrentRole"] = True

In [ ]:
# Covert start and end date to datetime format
df["DateStartRole"] = df["DateStartRole"].astype(np.datetime64)
df["DateEndRole"] = df["DateEndRole"].astype(np.datetime64)

In [ ]:
# Calculate the role duration. Note that we cannot use this directly, because that is what we're trying to predict.
# But this is a known and usable value once a role has ended. If there is a trend, it may continue.
df['RoleDuration'] = df['DateEndRole']-df['DateStartRole']

In [ ]:
# https://www.kaggle.com/cdeotte/xgb-fraud-with-magic-0-9600#Feature-Engineeringhttps://www.kaggle.com/cdeotte/xgb-fraud-with-magic-0-9600#Feature-Engineering

In [ ]:

def expand_rows(df):
    """
    Expand the dataframe to have a single row
    correspond to one month
    """
    df['RoleId'] = df.index.values.tolist()

    df.drop(df[df["DateEndRole"] == "N"].index, inplace=True)
    df.drop(df[df["DateStartRole"] == "N"].index, inplace=True)

    MaxDate = np.unique(df["DateEndRole"])[-2]
    length = df[df["DateEndRole"] == "C"].shape[0]
    print(f'Latest date in the dataset applied to {length} rows: {MaxDate}')
    df.loc[df["DateEndRole"] == "C","DateEndRole"] = MaxDate
    
    df["DateStartRole"] = df["DateStartRole"].astype(np.datetime64)
    df["DateEndRole"] = df["DateEndRole"].astype(np.datetime64)
    
    df = df.melt(id_vars=['RoleId', 'CompanyId', 'DirectorId', 'DirectorName', 'CompanyName', 'RoleName', 'Seniority'],value_name='Date',var_name='DateType')
    df = df.drop_duplicates(subset=['RoleId', 'Date'], keep=False)
    
    df = df.groupby('RoleId').apply(lambda x: x.set_index('Date').resample('M').pad())

    df = df.droplevel(level=0, axis=0)
    df.reset_index(inplace=True)

    return df

In [7]:

np.timedelta64(12, "M") <= np.timedelta64(1, "Y") 

True

In [ ]:
# Create a response variable set to TRUE if there is a turnover event within the next 12 months
df["time_left"] = df["DateEndRole"] - df["Date"] 
df["Turnover"] = [True if tenure > np.timedelta64(1, "Y") else False for tenure in df['time_left']]
df[(df.time_left <= np.timedelta64(11, "M")) & (df.current_role == True)]

In [ ]:
# We shift the role duration so that it is only at the end of the role that we start counting past roles
df1['PreviousRoleDuration'] = df1.groupby(['RoleId'])['RoleDuration'].shift(1)
df1.loc[pd.isnull(df1['PreviousRoleDuration']), 'PreviousRoleDuration'] = np.timedelta64(0, "D")

In [ ]:
#  We need to sort values again based on Director, Date, before we sum previous role duration to derive career length and company tenure
df1.sort_values(['DirectorId', 'Date'], inplace = True)
df1['CareerLength'] = df1.groupby('DirectorId')['PreviousRoleDuration'].transform(pd.Series.cumsum)
df1['CompanyTenure'] = df1.groupby(['DirectorId', 'CompanyId'])['PreviousRoleDuration'].transform(pd.Series.cumsum)

In [ ]:
# We create a PastRolesIncrementer as well as an ActiveRolesIncrementer to be used to tally the sum of past and active roles
df1['PastRolesIncrementer'] = [0 if DateType == 'DateStartRole' else 1 for DateType in df1['DateType']]
df1['ActiveRolesIncrementer'] = [1 if val == 'DateStartRole' else -1 for val in df1['DateType']]

In [ ]:
# Calculate the number of previous roles career-wide
df1.sort_values(['DirectorId', 'Date', 'DateType'], ascending=[True,True,False], inplace = True)
df1['PastRoles'] = df1.groupby(['DirectorId'])['PastRolesIncrementer'].transform(pd.Series.cumsum)

In [ ]:
# Calculate NetActiveRolesIncrementer which aggregates all role changes in a given month into a single value
df1['NetActiveRolesIncrementer'] = df1.groupby(['DirectorId', 'Date'])['ActiveRolesIncrementer'].transform('sum')

In [ ]:
# Set dummy variable equal to true if role is a CEO role
df1['CEO'] = [True if "CEO" in RoleName else False for RoleName in df1['RoleName']]

In [ ]:
# Drop duplicates where the Start Date equals the End Date 
df2 = df1.drop_duplicates(subset=['RoleId', 'Date'], keep=False)
# df2.drop_duplicates(subset=['DirectorId', 'Date'], keep=False, inplace=True)

# Resampling to create, for each role, a separate row for each month in the date range
df2 = df2.groupby('RoleId').apply(lambda x: x.set_index('Date').resample('M').pad())

In [ ]:
# Calculate role tenure for CEOs
df3 = df2[df2.CEO]
df3.drop_duplicates(subset=['CompanyId', 'DirectorId', 'CEO', 'Date'], keep=False, inplace=True)
df3['RoleTenure'] = df3.groupby('RoleId')['RoleId'].cumcount()

In [ ]:
# Count the tenure in months
df2.sort_values(['DirectorId', 'RoleId'], ascending=[True,True], inplace = True)
df2['RoleTenure'] = df2.groupby(['RoleId']).transform(Pd.Series.cumcount)

In [ ]:
df1_CEO = df1[df1.CEO]
# df1_CEO.drop_duplicates(subset=['CompanyId', 'DirectorId', 'Date'])
df1_CEO.drop_duplicates(subset=['CompanyId', 'DirectorId', 'CEO', 'Date'], keep=False, inplace=True)

# Calculate ActiveRoles. ATTENTION: do this only after removing duplicate dates or you risk double counting
df1['ActiveRoles'] = df1.groupby(['DirectorId'])['NetActiveRolesIncrementer'].transform(pd.Series.cumsum)

df1_CEO.sort_values(['CompanyName', 'Date'], inplace = True)
df1_CEO['PreviousCEORoleDuration'] = df1_CEO.groupby(['CompanyName'])['RoleDuration'].shift(2)

In [ ]:
def partition_df(dict, n):

    if (n == 1):
        return dict

    else: 
        p = sum(dict.values())/2
        _sum = 0
        _dict = {}

        while _sum < p:
            removed_item = dict.popitem()
            _dict[removed_item[0]] = removed_item[1]
            _sum += removed_item[1]
            
        return (partition_df(_dict, n/2), partition_df(dict, n/2))

keyz = partition_df(randd, 4)


In [ ]:
from multiprocessing import Pool

def parallelize_dataframe(df, func, n_cores=8):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

def add_tenure_cols(df):
    df["OneYearTenure"] = [1 if tenure > np.timedelta64(1, "Y") else 0 for tenure in df['RoleTenure']]
    return df

%time
df04 = parallelize_dataframe(df03, add_tenure_cols)